In [11]:
#import the library
#%matplotlib inline
import numpy as np
from sklearn import cross_validation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

In [12]:
#import the needed data
trainData=np.genfromtxt('train.csv',delimiter=',')
testData=np.genfromtxt('test.csv',delimiter=',')
trainingPixmaps=trainData[:,9:438]/255
trainingClasses=trainData[:,1]

In [13]:
#import the test data for prediction
testPixmaps = testData[:,9:438]/255
testIds = testData[:,0]

In [14]:
#cross validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split (trainingPixmaps, trainingClasses, 
                                                                      test_size=0.25, random_state=0)

In [15]:
#chagne the format of y_train and y_test
def changeY(ys) :
    newYs = []
    for i in range (0,len(ys)):
        index = int(ys[i])
        newPoint = np.zeros(98)
        newPoint[index] = 1
        newYs.append(newPoint)
    return np.asarray(newYs)

In [16]:
#define get batch method for later use
def getBatch(num) :
    batch_xs = []
    batch_ys = []
    for i in range (0, num) :
        index = random.randrange(0, len(X_train))
        batch_xs.append(X_train[index])
        batch_ys.append(y_train[index])
    return (np.asarray(batch_xs), changeY(np.asarray(batch_ys)))

In [17]:
#defin computing accuracy method
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

#define weight method
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

#define bias method
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#define convolutional method
def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#define pooling method
def max_pool_2x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#13x33
xs = tf.placeholder(tf.float32,[None,429])


#98 classes
ys = tf.placeholder(tf.float32,[None,98]) 

#weight
W = tf.Variable(tf.zeros([429,98]))
#bias
b = tf.Variable(tf.zeros([98]))

keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, 13, 33, 1])

In [18]:
## conv1 layer ##
W_conv1 = weight_variable([5,5, 1,18]) # patch 5x5, in size 1, out size 18
b_conv1 = bias_variable([18])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # output size 13x33x32
h_pool1 = max_pool_2x2(h_conv1) # output size 7x17x32

## conv2 layer ##
W_conv2 = weight_variable([3,3, 18, 64]) # patch 3x3, in size 18, out size 64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 7x17x64
h_pool2 = max_pool_2x2(h_conv2)     # output size 4x9x64



## func2 layer ##
W_fc2 = weight_variable([4*9*64, 2500])
b_fc2 = bias_variable([2500])
h_pool3_flat2 = tf.reshape(h_pool2, [-1,4*9*64])
h_fc2 = tf.nn.relu(tf.matmul(h_pool3_flat2, W_fc2) + b_fc2)
h_fc1_drop3 = tf.nn.dropout(h_fc2, keep_prob)

## func2 layer ##
W_fc3 = weight_variable([2500, 98])
b_fc3 = bias_variable([98])
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop3, W_fc3) + b_fc3)


# the error between prediction and real data
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))       # loss
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(ys,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
# important step
sess.run(tf.initialize_all_variables())



In [19]:
for i in range(11500):
    #print(mnist.train.next_batch)
    batch_xs, batch_ys = getBatch(50)
    
    #print(batch_ys)
    sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: 0.5})
    if i % 1150 == 0:
        print(compute_accuracy(
         X_test, changeY(y_test)))

0.00938668
0.611884
0.643863
0.651181
0.664227
0.670909
0.678148
0.678705
0.684273
0.682841


In [20]:
 y_pre = sess.run(prediction, feed_dict={xs: testPixmaps, keep_prob: 1})

In [21]:
list1 = []
list1 = []
for i in range(0,len(testPixmaps)):
    list1.append(np.argmax(y_pre[i]))
#print(testPixmaps)

In [22]:
results = []
for i in range(0,len(testPixmaps)):
    temp = []
    temp.append(int(testIds[i]))
    temp.append(int(list1[i]))
    results.append(temp)

In [23]:
import csv 

with open("finalone.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(results)